In [ ]:
# https://llamahub.ai/l/tools/llama-index-tools-neo4j

In [1]:
import os

from dotenv import load_dotenv

In [2]:
## Loading Environment Variables
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DB = os.getenv("NEO4J_DB")

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

NEO4J_AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)
INDEX_NAME = "vector_index"

In [3]:
from neo4j import GraphDatabase

In [4]:
## Connecting to Neo4J instance
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

## Retrieval from the Graph

In [5]:
from llama_index.llms.google_genai import GoogleGenAI

from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from google.genai.types import EmbedContentConfig

EMBEDDING_SIZE = 768

embedding_config = EmbedContentConfig(output_dimensionality=EMBEDDING_SIZE)
embed_model = GoogleGenAIEmbedding(
    model_name="models/gemini-embedding-001",
    embed_batch_size=100,
    embedding_config=embedding_config,
    api_key=GEMINI_API_KEY
)

llm = GoogleGenAI(model = "gemini-2.5-flash-lite", api_key=GEMINI_API_KEY, temperature=0)

## RAG

In [27]:
import templates

import nest_asyncio
nest_asyncio.apply()

In [17]:
QUERY = "What is the relationship between the characters Harry Potter and Lord Voldemort"

In [8]:
def vector_search(query: str):
    query_embedding = embed_model.get_query_embedding(query)
    
    result = driver.execute_query('''
        CALL db.index.vector.queryNodes($index, 5, $queryEmbedding)
        YIELD node, score
        RETURN node.text AS text, score
        ''', index=INDEX_NAME,queryEmbedding=query_embedding,
        database_=NEO4J_DB)
    
    return result

In [9]:
result = vector_search(QUERY)
result

EagerResult(records=[<Record text='== Films ==\n\n\n=== Harry Potter and the Philosopher\'s Stone (2001) ===\n\nHarry Potter is an orphaned boy brought up by his unkind Muggle (non-magical) aunt and uncle. At the age of eleven, half-giant Rubeus Hagrid informs him that he is actually a wizard and that his parents were murdered by an evil wizard named Lord Voldemort. Voldemort also attempted to kill one-year-old Harry on the same night, but his killing curse mysteriously rebounded and reduced him to a weak and helpless form. Harry became extremely famous in the Wizarding World as a result. Harry begins his first year at Hogwarts School of Witchcraft and Wizardry and learns about magic. During the year, Harry and his friends Ron Weasley and Hermione Granger become entangled in the mystery of the Philosopher\'s Stone which is being kept within the school.\n\n\n=== Harry Potter and the Chamber of Secrets (2002) ===\n\nHarry, Ron, and Hermione return to Hogwarts for their second year, which

In [10]:
rag_context = [item[0] for item in result[0]]

In [11]:
def rag_chain(query: str, rag_context):
    graph_rag_prompt = templates.rag_prompt.format(
        rag_context=rag_context,
        query=query
    )

    response = llm.complete(graph_rag_prompt)

    return response.text

In [ ]:
rag_response = rag_chain(QUERY, rag_context)

In [13]:
print(rag_response)

Harry Potter and Lord Voldemort are arch-enemies. Voldemort murdered Harry's parents and attempted to kill Harry as a baby, but the killing curse rebounded, weakening Voldemort. Harry became famous in the Wizarding World for surviving this attack. Throughout the series, Voldemort's goal is to kill Harry, and Harry's quest is to overcome him.


## Graph Search

In [35]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore

graph_store = Neo4jGraphStore(
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    url=NEO4J_URI,
    database=NEO4J_DB
)

graph_schema = graph_store.schema

In [37]:
def query_to_cypher(query: str):
    prompt = f"""
    Task: Generate Cypher queries for the user query,
    to query a Neo4j graph database based on the provided schema definition.
    
    user query:
    {query}
    
    Instructions:
    Use only the provided relationship types and properties.
    Do not use any other relationship types or properties that are not provided.
    If you cannot generate a Cypher statement based on the provided schema, explain the reason to the user.
    
    Schema:
    {graph_schema}

    Note: Do not include any explanations or apologies in your responses.
    """
    
    response = llm.complete(prompt)
    cypher_query = response.text.removeprefix("```cypher")
    cypher_query = cypher_query.removesuffix("```")
    
    return cypher_query

In [38]:
cypher = query_to_cypher(QUERY)
cypher

'MATCH (c1:CHARACTER {name: "Harry Potter"})-[r]->(c2:CHARACTER {name: "Lord Voldemort"}) RETURN c1, r, c2'

In [40]:
result = driver.execute_query(cypher)
result[0]

[<Record c1=<Node element_id='4:d36de59f-21a7-45a5-b3ab-0cf151118530:233' labels=frozenset({'CHARACTER'}) properties={'name': 'Harry Potter', 'node_id': '31'}> r=<Relationship element_id='5:d36de59f-21a7-45a5-b3ab-0cf151118530:6919945754198933741' nodes=(<Node element_id='4:d36de59f-21a7-45a5-b3ab-0cf151118530:233' labels=frozenset({'CHARACTER'}) properties={'name': 'Harry Potter', 'node_id': '31'}>, <Node element_id='4:d36de59f-21a7-45a5-b3ab-0cf151118530:237' labels=frozenset({'CHARACTER'}) properties={'name': 'Lord Voldemort', 'node_id': '35'}>) type='PARENTS_MURDERED_BY' properties={'details': 'his parents were murdered by an evil wizard named Lord Voldemort.'}> c2=<Node element_id='4:d36de59f-21a7-45a5-b3ab-0cf151118530:237' labels=frozenset({'CHARACTER'}) properties={'name': 'Lord Voldemort', 'node_id': '35'}>>,
 <Record c1=<Node element_id='4:d36de59f-21a7-45a5-b3ab-0cf151118530:253' labels=frozenset({'CHARACTER'}) properties={'name': 'Harry Potter', 'description': 'An orphaned 

In [82]:
graph_result = []
for item in result[0]:
    relationship = f"""{item[0].get("name")} - {item[1].type} - {item[2].get("name")}"""
    metadetails = f"""start node: {list(item[0].items())}, node label: {list(item[0].labels)[0]}
    relationship: {item[1].type}, details: {list(item[1].items())}
    end node: {list(item[2].items())}, node label: {list(item[2].labels)[0]}"""
    graph_result.append((relationship, metadetails))

In [84]:
for result in graph_result:
    print(result[0])

Harry Potter - PARENTS_MURDERED_BY - Lord Voldemort
Harry Potter - PARENTS_MURDERED_BY - Lord Voldemort
Harry Potter - ENCOUNTERED - Lord Voldemort
Harry Potter - WORK_TOGETHER_TO_DESTROY - Lord Voldemort
Harry Potter - CONNECTION_WITH_BECOMING_STRONGER - Lord Voldemort


In [85]:
def record_to_result(record):
    graph_result = []
    for item in record:
        relationship = f"""{item[0].get("name")} - {item[1].type} - {item[2].get("name")}"""
        metadetails = f"""start node: {list(item[0].items())}, node label: {list(item[0].labels)[0]}
        relationship: {item[1].type}, details: {list(item[1].items())}
        end node: {list(item[2].items())}, node label: {list(item[2].labels)[0]}"""
        graph_result.append((relationship, metadetails))
    
    return graph_result

In [86]:
def graph_chain(query: str):
    cypher_query = query_to_cypher(query)
    try:
        result = driver.execute_query(cypher)
        graph_result = record_to_result(result[0])
        
        return graph_result
    except Exception as e:
        print(f"An error has occurred: {e}")

## GraphRAG

In [93]:
def graph_rag_chain(query: str):
    graph_context = graph_chain(query)
    
    graph_rag_prompt = templates.graph_rag_prompt.format(
        rag_context=rag_context,
        graph_context=graph_context,
        query=query
    )

    response = llm.complete(graph_rag_prompt)

    return response.text

In [94]:
graph_rag_response = graph_rag_chain(QUERY)
graph_rag_response

"Harry Potter's parents were murdered by Lord Voldemort. Harry also had a terrifying encounter with a reborn Lord Voldemort, and their connection grew stronger. Dumbledore and Harry worked together to destroy Lord Voldemort."